In [10]:
import pandas as pd
import cufflinks as cf
import mytools.date as dt
import mytools.plot as mpl

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

## Italian intensive care cases 

In [11]:
# loading the data
df = pd.read_csv('italy-intensive_care.csv')
a = list((dt.day_of_year_to_date(v)).strftime("%d/%m/%y") for v in df['day'])
df.index = a
df.drop(columns=['day'])
# display the table


,intensive care
24/02/20,27
25/02/20,35
26/02/20,36
27/02/20,56
28/02/20,64
29/02/20,105
01/03/20,140
02/03/20,166
03/03/20,229
04/03/20,295


## Fitting intensive care data with exponential and logistic models

The exponential model is:

$$y = e^{k(x-x_0)} + y_0$$

where $k, x_0, y_0$ are the parameters to estimate.

For the logistic model:

$$y = \frac{c}{1+e^{-k(x-x_0)}} + y_0$$

where $c, k, x_0, y_0$ are the parameters to estimate.

In [12]:
fig = mpl.iplot_analysis_plot(df.drop(columns=['day']), title='Intensive care cases')
fig.show()

[0.65       0.46165034 1.        ]
`ftol` termination condition is satisfied.
Function evaluations 15, initial cost 1.1284e+01, final cost 4.9479e-03, first-order optimality 4.52e-08.
            Exponential model
            x0 = -41.35908305089964
            y0 = -757.8637157709605
            k = 0.06679991477105318
            
`gtol` termination condition is satisfied.
Function evaluations 26, initial cost 2.9527e+00, final cost 2.0395e-04, first-order optimality 2.77e-09.
        Sigmoid model
        x0 = 77.97241530947399
        y0 = -100.20065320312524
        c = 4723.60495362461
        k = 0.16936067206548958
        asymptot = 4623.404300421485
        flex = 77.97241530947399, 2261.60182360918
        
`ftol` termination condition is satisfied.
Function evaluations 35, initial cost 1.1689e+00, final cost 3.5727e-04, first-order optimality 1.05e-08.
        Sigmoid derivative model
        x0 = 87.4465688161983
        y0 = -157.8695904931957
        c = 103472.980399675